# Exploration des données de tweets

**Auteur :** Mehdi MUNIM

**Date :** 2023-11-26

**Description :**

Ce notebook explore les données de tweets afin de mieux comprendre leurs caractéristiques et de préparer les étapes de prétraitement et de modélisation.

In [ ]:
#@title 1. Importation des librairies

# Librairies pour la manipulation et l'analyse des données
import pandas as pd
import numpy as np

# Librairies pour le traitement du langage naturel (NLP)
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Librairies pour la visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Librairies pour la sauvegarde des données
import os
import pickle

# Téléchargement des ressources NLTK
nltk.download('stopwords')
nltk.download('wordnet')

## 2. Chargement et aperçu des données

In [ ]:
#@title 2.1 Chargement des données

# Téléchargement et décompression du fichier zip
!wget "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+7%20-%20D%C3%A9tectez%20les%20Bad%20Buzz%20gr%C3%A2ce%20au%20Deep%20Learning/sentiment140.zip"
!unzip sentiment140.zip

# Charger les données du fichier CSV
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)

In [ ]:
#@title 2.2 Aperçu des données

# Afficher les premières lignes du DataFrame
print("Premières lignes du DataFrame :")
display(df.head())

In [ ]:
#@title 2.3 Informations sur les données

# Afficher les noms de colonnes
print("\nNoms de colonnes :")
print(df.columns)

# Afficher les types de données et les informations manquantes
print("\nInformations sur le DataFrame :")
df.info()

In [ ]:
#@title 2.4 Statistiques descriptives

# Afficher les statistiques descriptives
print("\nStatistiques descriptives :")
display(df.describe())

## 3. Analyse exploratoire

In [ ]:
#@title 3.1 Distribution des classes

# Afficher la distribution des sentiments (0 = négatif, 4 = positif)
sns.countplot(x=0, data=df)
plt.title('Distribution des sentiments')
plt.show()

In [ ]:
#@title 3.1 Distribution des classes

# Afficher la distribution des sentiments (0 = négatif, 4 = positif)
sns.countplot(x=0, data=df)
plt.title('Distribution des sentiments')
plt.show()

In [ ]:
#@title 3.2 Longueur des tweets

# Calculer la longueur des tweets
df['longueur_tweet'] = df[5].apply(len)

# Afficher la distribution de la longueur des tweets
sns.histplot(df['longueur_tweet'], bins=50)
plt.title('Distribution de la longueur des tweets')
plt.xlabel('Longueur du tweet')
plt.ylabel('Nombre de tweets')
plt.show()

In [ ]:
#@title 3.3 Fréquence des mots

from collections import Counter

# Calculer la fréquence des mots
all_words = ' '.join(df[5].tolist()).split()
word_counts = Counter(all_words)

# Afficher les 20 mots les plus fréquents
print("20 mots les plus fréquents :")
print(word_counts.most_common(20))

In [ ]:
#@title 3.4 Nuage de mots

from wordcloud import WordCloud

# Créer un nuage de mots
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_counts)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Nuage de mots')
plt.show()

## 4. Conclusion

Dans ce notebook, nous avons exploré les données de tweets en effectuant une analyse descriptive et une analyse exploratoire. 

Nous avons notamment observé :

* La distribution des sentiments (classes équilibrées).
* La longueur des tweets (distribution asymétrique).
* La fréquence des mots (mots les plus fréquents).
* Un nuage de mots pour visualiser les termes importants.

Ces analyses nous permettent de mieux comprendre les données et de préparer les étapes de prétraitement et de modélisation.
